# Poblar BD de Neo4j

In [55]:
from neo4j import GraphDatabase
import csv

# Neo4j connection details
uri = "bolt://localhost:7687"
username = "neo4j"
password = "postgres"

# CSV file path
csv_file = "nodes.csv"

driver = GraphDatabase.driver(uri, auth=(username, password))

# Cypher query to create nodes and relationships
create_query = """
LOAD CSV WITH HEADERS FROM 'file:///{csv_file}' AS row
FOREACH (_ IN CASE WHEN row.type = 'person' THEN [1] ELSE [] END |
    CREATE (n:person)
    SET n.type = row.type,
        n.country = row.country,
        n.id = row.id
)

"""

# Connect to Neo4j


# Open a Neo4j session
with driver.session() as session:
    # Execute the Cypher query to create nodes and relationships
    session.run(create_query.format(csv_file=csv_file))


# Cypher query to create nodes and relationships
create_query = """
LOAD CSV WITH HEADERS FROM 'file:///{csv_file}' AS row
FOREACH (_ IN CASE WHEN row.type = 'organization' THEN [1] ELSE [] END |
    CREATE (n:organization)
    SET n.type = row.type,
        n.country = row.country,
        n.id = row.id
)
"""


# Open a Neo4j session
with driver.session() as session:
    # Execute the Cypher query to create nodes and relationships
    session.run(create_query.format(csv_file=csv_file))
    
    
# Cypher query to create nodes and relationships
create_query = """
LOAD CSV WITH HEADERS FROM 'file:///{csv_file}' AS row
FOREACH (_ IN CASE WHEN row.type = 'company' THEN [1] ELSE [] END |
    CREATE (n:company)
    SET n.type = row.type,
        n.country = row.country,
        n.id = row.id
)
"""

# Open a Neo4j session
with driver.session() as session:
    # Execute the Cypher query to create nodes and relationships
    session.run(create_query.format(csv_file=csv_file))
    

# Cypher query to create nodes and relationships
create_query = """
LOAD CSV WITH HEADERS FROM 'file:///{csv_file}' AS row
FOREACH (_ IN CASE WHEN row.type = 'political_organization' THEN [1] ELSE [] END |
    CREATE (n:political_organization)
    SET n.type = row.type,
        n.country = row.country,
        n.id = row.id
)
"""

# Open a Neo4j session
with driver.session() as session:
    # Execute the Cypher query to create nodes and relationships
    session.run(create_query.format(csv_file=csv_file))
    

# Cypher query to create nodes and relationships
create_query = """
LOAD CSV WITH HEADERS FROM 'file:///{csv_file}' AS row
FOREACH (_ IN CASE WHEN row.type = 'vessel' THEN [1] ELSE [] END |
    CREATE (n:vessel)
    SET n.type = row.type,
        n.country = row.country,
        n.id = row.id
)
"""

# Open a Neo4j session
with driver.session() as session:
    # Execute the Cypher query to create nodes and relationships
    session.run(create_query.format(csv_file=csv_file))


# Cypher query to create nodes and relationships
create_query = """
LOAD CSV WITH HEADERS FROM 'file:///{csv_file}' AS row
FOREACH (_ IN CASE WHEN row.type = 'location' THEN [1] ELSE [] END |
    CREATE (n:location)
    SET n.type = row.type,
        n.country = row.country,
        n.id = row.id
)
"""

# Open a Neo4j session
with driver.session() as session:
    # Execute the Cypher query to create nodes and relationships
    session.run(create_query.format(csv_file=csv_file))
    

# Cypher query to create nodes and relationships
create_query = """
LOAD CSV WITH HEADERS FROM 'file:///{csv_file}' AS row
FOREACH (_ IN CASE WHEN row.type = 'notype' THEN [1] ELSE [] END |
    CREATE (n:notype)
    SET n.type = row.type,
        n.country = row.country,
        n.id = row.id
)
"""

# Open a Neo4j session
with driver.session() as session:
    # Execute the Cypher query to create nodes and relationships
    session.run(create_query.format(csv_file=csv_file))
    
create_query = """
LOAD CSV WITH HEADERS FROM 'file:///{csv_file}' AS row
FOREACH (_ IN CASE WHEN row.type = 'event' THEN [1] ELSE [] END |
    CREATE (n:event)
    SET n.type = row.type,
        n.country = row.country,
        n.id = row.id
)
"""

# Open a Neo4j session
with driver.session() as session:
    # Execute the Cypher query to create nodes and relationships
    session.run(create_query.format(csv_file=csv_file))

create_query = """
LOAD CSV WITH HEADERS FROM 'file:///{csv_file}' AS row
FOREACH (_ IN CASE WHEN row.type = 'movement' THEN [1] ELSE [] END |
    CREATE (n:movement)
    SET n.type = row.type,
        n.country = row.country,
        n.id = row.id
)
"""

# Open a Neo4j session
with driver.session() as session:
    # Execute the Cypher query to create nodes and relationships
    session.run(create_query.format(csv_file=csv_file))


# Close the Neo4j driver
driver.close()

In [59]:
from neo4j import GraphDatabase
import csv

# Neo4j connection details
uri = "bolt://localhost:7687"
username = "neo4j"
password = "postgres"

# CSV file path
csv_file = "links.csv"

# Cypher query to create relationships
create_query = """
LOAD CSV WITH HEADERS FROM 'file:///{csv_file}' AS row
MATCH (source)
WHERE source.id = row.source AND row.type = 'ownership' AND (source:organization OR source:company OR source:person OR source:political_organization OR source:vessel OR source:location OR source:notype OR source:event OR source:movement)
MATCH (target)
WHERE target.id = row.target AND (target:organization OR target:company OR target:person OR target:political_organization OR target:vessel OR target:location OR target:notype OR target:event OR target:movement)
CREATE (source)-[:ownership]->(target)
"""

# Connect to Neo4j
driver = GraphDatabase.driver(uri, auth=(username, password))

# Open a Neo4j session
with driver.session() as session:
    # Execute the Cypher query to create relationships
    session.run(create_query.format(csv_file=csv_file))

# Cypher query to create relationships
create_query = """
LOAD CSV WITH HEADERS FROM 'file:///{csv_file}' AS row
MATCH (source)
WHERE source.id = row.source AND row.type = 'membership' AND (source:organization OR source:company OR source:person OR source:political_organization OR source:vessel OR source:location OR source:notype OR source:event OR source:movement)
MATCH (target)
WHERE target.id = row.target AND (target:organization OR target:company OR target:person OR target:political_organization OR target:vessel OR target:location OR target:notype OR target:event OR target:movement)
CREATE (source)-[:membership]->(target)
"""

# Open a Neo4j session
with driver.session() as session:
    # Execute the Cypher query to create relationships
    session.run(create_query.format(csv_file=csv_file))
    
# Cypher query to create relationships
create_query = """
LOAD CSV WITH HEADERS FROM 'file:///{csv_file}' AS row
MATCH (source)
WHERE source.id = row.source AND row.type = 'family_relationship' AND (source:organization OR source:company OR source:person OR source:political_organization OR source:vessel OR source:location OR source:notype OR source:event OR source:movement)
MATCH (target)
WHERE target.id = row.target AND (target:organization OR target:company OR target:person OR target:political_organization OR target:vessel OR target:location OR target:notype OR target:event OR target:movement)
CREATE (source)-[:family_relationship]->(target)
"""

# Open a Neo4j session
with driver.session() as session:
    # Execute the Cypher query to create relationships
    session.run(create_query.format(csv_file=csv_file))


# Cypher query to create relationships
create_query = """

LOAD CSV WITH HEADERS FROM 'file:///{csv_file}' AS row
MATCH (source)
WHERE source.id = row.source AND row.type = 'partnership' AND (source:organization OR source:company OR source:person OR source:political_organization OR source:vessel OR source:location OR source:notype OR source:event OR source:movement)
MATCH (target)
WHERE target.id = row.target AND (target:organization OR target:company OR target:person OR target:political_organization OR target:vessel OR target:location OR target:notype OR target:event OR target:movement)
CREATE (source)-[:partnership]->(target)
"""

# Open a Neo4j session
with driver.session() as session:
    # Execute the Cypher query to create relationships
    session.run(create_query.format(csv_file=csv_file))
# Close the Neo4j driver
driver.close()

# Gráfico Nodos Sospechosos con sus Conexiones de Nivel 1

In [84]:
from neo4j import GraphDatabase
from pyvis.network import Network
import networkx as nx

# Connect to the Neo4j database
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "postgres"))

# Define the Cypher query
cypher_query = """
MATCH (node)-[r:ownership|membership|partnership|family_relationship]-(n)
WHERE node.id = "Mar de la Vida OJSC" OR
	    node.id = "979893388" OR
        node.id = "Oceanfront Oasis Inc Carriers" OR
        node.id = "8327"
RETURN node, r, n
"""

# Execute the query and retrieve the graph data
with driver.session() as session:
    result = session.run(cypher_query)
    graph_data = [(record["node"]["id"], record["n"]["id"], record["r"].type) for record in result]

# Create a Pyvis Network object
network = Network(height="500px", width="100%", notebook=True, cdn_resources='remote')

# Add nodes and edges to the network
for source, target, relation_type in graph_data:
    if source == "Mar de la Vida OJSC":
        network.add_node(source,title = source, shape = "star" ,color = "navy")
    elif source == "979893388":
        network.add_node(source,title = source, shape = "star" ,color = "green")
    elif source == "Oceanfront Oasis Inc Carriers":
        network.add_node(source,title = source, shape = "star" ,color = "orange")
    elif source == "8327":
        network.add_node(source,title = source, shape = "star" ,color = "purple")    
    else:
        network.add_node(source, title = source)
    if target == "Mar de la Vida OJSC":
        network.add_node(target,title = target, shape = "star" ,color = "navy")
    elif target == "979893388":
        network.add_node(target,title = target, shape = "star" ,color = "green")
    elif target == "Oceanfront Oasis Inc Carriers":
        network.add_node(target,title = target, shape = "star" ,color = "orange")
    elif target == "8327":
        network.add_node(target,title = target, shape = "star" ,color = "purple")    
    else:
        network.add_node(target,title = target)
    network.add_edge(source, target, title=relation_type)

# Visualize the network
network.show("2_Graph_Entities_Ilegal.html")

# Close the Neo4j driver connection
driver.close()

2_Graph_Entities_Ilegal.html


# Gráfico Nodos Sospechosos con Conexiones de Nivel 1 y 2 entre ellos

In [85]:
from neo4j import GraphDatabase
from pyvis.network import Network
import networkx as nx

# Connect to the Neo4j database
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "postgres"))

# Define the Cypher query
cypher_query = """
MATCH (node)-[rel1]-(lvl1)-[rel2]-(lvl2)
WHERE (node.id = "Mar de la Vida OJSC" OR
        node.id = "979893388" OR
        node.id = "Oceanfront Oasis Inc Carriers" OR
        node.id = "8327") AND
        (lvl2.id = "Mar de la Vida OJSC" OR
        lvl2.id = "979893388" OR
        lvl2.id = "Oceanfront Oasis Inc Carriers" OR
        lvl2.id = "8327" OR
        lvl1.id = "Mar de la Vida OJSC" OR
        lvl1.id = "979893388" OR
        lvl1.id = "Oceanfront Oasis Inc Carriers" OR
        lvl1.id = "8327")
        AND (node.id <> lvl1.id AND node.id <> lvl2.id AND lvl1.id <> lvl2.id)
RETURN node, rel1, lvl1, rel2, lvl2
"""

# Execute the query and retrieve the graph data
with driver.session() as session:
    result = session.run(cypher_query)
    graph_data = [(record["node"]["id"],record["node"]["type"],record["lvl1"]["id"], record["lvl1"]["type"],record["lvl2"]["id"], record["lvl2"]["type"], record["rel1"].type, record["rel2"].type) for record in result]

# Create a Pyvis Network object
network = Network(height="500px", width="100%", notebook=True, cdn_resources='remote')

# Add nodes and edges to the network
for source, sourcet, level1, level1t, target, targett, rela1, rela2, in graph_data:
    #if source != "979893388" and target != "979893388" and level1 != "979893388" and level2 != "979893388":
    #   if source in ["Mar de la Vida OJSC","979893388","Oceanfront Oasis Inc Carriers","8327"] or target in ["Mar de la Vida OJSC","979893388","Oceanfront Oasis Inc Carriers","8327"]:
        if source == "Mar de la Vida OJSC":
            network.add_node(source,title = sourcet, shape = "star" ,color = "navy")
        elif source == "979893388":
            network.add_node(source,title = sourcet, shape = "star" ,color = "green")
        elif source == "Oceanfront Oasis Inc Carriers":
            network.add_node(source,title = sourcet, shape = "star" ,color = "orange")
        elif source == "8327":
            network.add_node(source,title = sourcet, shape = "star" ,color = "purple")    
        else:
            network.add_node(source, title = sourcet)
        if level1 == "Mar de la Vida OJSC":
            network.add_node(level1,title = level1t, shape = "star" ,color = "navy")
        elif level1 == "979893388":
            network.add_node(level1,title = level1t, shape = "star" ,color = "green")
        elif level1 == "Oceanfront Oasis Inc Carriers":
            network.add_node(level1,title = level1t, shape = "star" ,color = "orange")
        elif level1 == "8327":
            network.add_node(level1,title = level1t, shape = "star" ,color = "purple")    
        else:
            network.add_node(level1,title = level1t)
        if target == "Mar de la Vida OJSC":
            network.add_node(target,title = targett, shape = "star" ,color = "navy")
            network.add_edge(level1, target, title=rela2)
        elif target == "979893388":
            network.add_node(target,title = targett, shape = "star" ,color = "green")
            network.add_edge(level1, target, title=rela2)
        elif target == "Oceanfront Oasis Inc Carriers":
            network.add_node(target,title = targett, shape = "star" ,color = "orange")
            network.add_edge(level1, target, title=rela2)
        elif target == "8327":
            network.add_node(target,title = targett, shape = "star" ,color = "purple")
            network.add_edge(level1, target, title=rela2)
        network.add_edge(source, level1, title=rela1)
        

# Visualize the network
network.show("3_Graph_lvl12.html")

# Close the Neo4j driver connection
driver.close()

3_Graph_lvl12.html


# Relaciones directas con 8327

In [122]:
from neo4j import GraphDatabase
from pyvis.network import Network
import networkx as nx

# Connect to the Neo4j database
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "postgres"))

# Define the Cypher query
cypher_query = """
MATCH (node)-[r:ownership|membership|partnership|family_relationship]-(n)
WHERE (node:organization OR node:company OR node:person OR node:political_organization OR node:vessel OR node:location OR node:notype OR node:event OR node:movement)
  AND (n:organization OR n:company OR n:person OR n:political_organization OR n:vessel OR n:location OR n:notype OR node:event OR node:movement)
  AND node.id IN ["Mar de la Vida OJSC", "979893388", "Oceanfront Oasis Inc Carriers", "8327"]
RETURN node, r, n
"""

# Execute the query and retrieve the graph data
with driver.session() as session:
    result = session.run(cypher_query)
    graph_data = [(record["node"]["id"], record["node"]["type"], record["n"]["id"], record["n"]["type"], record["r"].type) for record in result]

# Create a Pyvis Network object
network = Network(height="500px", width="100%", notebook=True, cdn_resources='remote')

# Colors
node_colors = {
    "â\x80\x9cIllegal": "black"
}

# Shapes
node_shapes = {
    "â\x80\x9cIllegal": "square"
}

# Add nodes and edges to the network
for source, sourcet, target, targett, rela in graph_data:
    #if source != "979893388" and target != "979893388" and level1 != "979893388" and level2 != "979893388":
    #   if source in ["Mar de la Vida OJSC","979893388","Oceanfront Oasis Inc Carriers","8327"] or target in ["Mar de la Vida OJSC","979893388","Oceanfront Oasis Inc Carriers","8327"]:
        if source == "8327":
            network.add_node(source,title = sourcet, shape = "star" ,color = "purple")    
            network.add_node(target,title = targett, shape = node_shapes.get(target,"dot"), color = node_colors.get(target,"lightblue"))
            network.add_edge(source, target, title=rela)
        

# Visualize the network
network.show("4_Graph_8327_ilegal.html")

# Close the Neo4j driver connection
driver.close()

4_Graph_8327_ilegal.html


# Gráfico Nods Sospechosos con Conexiones de Nivel 3 entre ellos

In [60]:
from neo4j import GraphDatabase
from pyvis.network import Network
import networkx as nx

# Connect to the Neo4j database
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "postgres"))

# Define the Cypher query
cypher_query = """
MATCH (node)-[rel1]-(lvl1)-[rel2]-(lvl2)-[rel3]-(lvl3)
WHERE (node.id = "Mar de la Vida OJSC" OR
        node.id = "979893388" OR
        node.id = "Oceanfront Oasis Inc Carriers" OR
        node.id = "8327") AND
        (lvl3.id = "Mar de la Vida OJSC" OR
        lvl3.id = "979893388" OR
        lvl3.id = "Oceanfront Oasis Inc Carriers" OR
        lvl3.id = "8327")
        AND (node.id <> lvl1.id AND node.id <> lvl2.id AND node.id <> lvl3.id AND lvl1.id <> lvl2.id AND lvl1.id <> lvl3.id AND lvl2.id <> lvl3.id )
RETURN node, rel1, lvl1, rel2, lvl2, rel3, lvl3
"""

# Execute the query and retrieve the graph data
with driver.session() as session:
    result = session.run(cypher_query)
    graph_data = [(record["node"]["id"], record["lvl1"]["id"],record["lvl2"]["id"],record["lvl3"]["id"], record["rel1"].type, record["rel2"].type, record["rel3"].type) for record in result]

# Create a Pyvis Network object
network = Network(height="500px", width="100%", notebook=True, cdn_resources='remote')

# Add nodes and edges to the network
for source, level1, level2, target, rela1, rela2, rela3 in graph_data:
    #if source != "979893388" and target != "979893388" and level1 != "979893388" and level2 != "979893388":
    #   if source in ["Mar de la Vida OJSC","979893388","Oceanfront Oasis Inc Carriers","8327"] or target in ["Mar de la Vida OJSC","979893388","Oceanfront Oasis Inc Carriers","8327"]:
        if source == "Mar de la Vida OJSC":
            network.add_node(source,title = source ,color = "navy")
        elif source == "979893388":
            network.add_node(source,title = source ,color = "green")
        elif source == "Oceanfront Oasis Inc Carriers":
            network.add_node(source,title = source ,color = "orange")
        elif source == "8327":
            network.add_node(source,title = source ,color = "purple")    
        else:
            network.add_node(source, title = source)
        if target == "Mar de la Vida OJSC":
            network.add_node(target,title = target ,color = "navy")
        elif target == "979893388":
            network.add_node(target,title = target ,color = "green")
        elif target == "Oceanfront Oasis Inc Carriers":
            network.add_node(target,title = target ,color = "orange")
        elif target == "8327":
            network.add_node(target,title = target ,color = "purple")    
        else:
            network.add_node(target,title = target)
        network.add_node(level1,title = level1)
        network.add_node(level2,title = level2)
        network.add_edge(source, level1, title=rela1)
        network.add_edge(level1, level2, title=rela2)
        network.add_edge(level2, target, title=rela3)

# Visualize the network
network.show("graph_lvl3.html")

# Close the Neo4j driver connection
driver.close()

graph_lvl3.html


# Gráfico Todas conexiones Nivel 1 sombreados por Categoría

In [123]:
from neo4j import GraphDatabase
from pyvis.network import Network

# Connect to the Neo4j database
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "postgres"))

# Define the Cypher query
cypher_query = """
MATCH (node)-[r:ownership|membership|partnership|family_relationship]-(n)
WHERE (node:organization OR node:company OR node:person OR node:political_organization OR node:vessel OR node:location OR node:notype OR node:event OR node:movement)
  AND (n:organization OR n:company OR n:person OR n:political_organization OR n:vessel OR n:location OR n:notype OR node:event OR node:movement)
  AND node.id IN ["Mar de la Vida OJSC", "979893388", "Oceanfront Oasis Inc Carriers", "8327"]
RETURN node, r, n
"""

# Execute the query and retrieve the graph data
with driver.session() as session:
    result = session.run(cypher_query)
    graph_data = [(record["node"]["id"], record["node"]["type"], record["n"]["id"], record["n"]["type"], record["r"].type) for record in result]

# Create a Pyvis Network object
network = Network(height="500px", width="100%", notebook=True, cdn_resources='remote')

# Define node colors based on source type
node_colors = {
    "organization": "navy",
    "company": "darkgreen",
    "person": "darkorange",
    "political_organization": "lightblue",
    "vessel": "magenta",
    "location": "purple",
    "notype": "darkgrey",
    "event": "cyan",
    "movement": "black"
}

# Add nodes and edges to the network
for source, source_type, target, target_type, relation_type in graph_data:
    if source in ["Mar de la Vida OJSC", "979893388", "Oceanfront Oasis Inc Carriers", "8327"]:
        network.add_node(source, title=source, color = "darkred", shape = "star")
    else:
        network.add_node(source, title=source, color=node_colors.get(source_type, "gray"))
    if target in ["Mar de la Vida OJSC", "979893388", "Oceanfront Oasis Inc Carriers", "8327"]:
        network.add_node(target, title=target, color = "darkred", shape = "star")
    else:
        network.add_node(target, title=target, color=node_colors.get(target_type, "gray"))
    network.add_edge(source, target, title=relation_type)

# Visualize the network
network.show("1_graph_categories1.html")

# Close the Neo4j driver connection
driver.close()

1_graph_categories1.html


# Ilegales Nivel 1

In [136]:
from neo4j import GraphDatabase
from pyvis.network import Network

# Connect to the Neo4j database
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "postgres"))

# Define the Cypher query
cypher_query = """
MATCH (node)-[r]-(n)
WHERE node.id IN ["â\x80\x9cillegalâ","â\x80\x9cIllegal","Dark Web Vendor Illegal Narcotics","Heroin Cocaine Exchange Bitcoin"]
RETURN node,r ,n
"""
#=~ '(?i).*illegal.*'

# Execute the query and retrieve the graph data
with driver.session() as session:
    result = session.run(cypher_query)
    graph_data = [(record["node"]["id"], record["node"]["type"], record["n"]["id"], record["n"]["type"], record["r"].type) for record in result]

# Create a Pyvis Network object
network = Network(height="500px", width="100%", notebook=True,cdn_resources='remote')

# Define node colors
node_colors = {
    "â\x80\x9cillegalâ": "black",
    "â\x80\x9cIllegal": "black",
    "Dark Web Vendor Illegal Narcotics": "black",
    "Heroin Cocaine Exchange Bitcoin": "black",
    "Mar de la Vida OJSC": "navy",
    "979893388": "green",
    "Oceanfront Oasis Inc Carriers": "orange",
    "8327": "purple"
}

# Define node shapes
node_shapes = {
    "â\x80\x9cillegalâ": "square",
    "â\x80\x9cIllegal": "square",
    "Dark Web Vendor Illegal Narcotics": "square",
    "Heroin Cocaine Exchange Bitcoin": "square",
    "Mar de la Vida OJSC": "star",
    "979893388": "star",
    "Oceanfront Oasis Inc Carriers": "star",
    "8327": "star"
}

# Add nodes and edges to the network
for source, source_type, target, target_type, relation_type in graph_data:
    network.add_node(source, title=source_type,shape=node_shapes.get(source,"dot"), color=node_colors.get(source, "lightblue"))
    network.add_node(target, title=target_type,shape=node_shapes.get(target,"dot") , color=node_colors.get(target, "lightblue"))
    network.add_edge(source, target, title=relation_type)

# Visualize the network
network.show("5_Illegal.html")

# Close the Neo4j driver connection
driver.close()

5_Illegal.html


# Ilegales Nivel 2

In [137]:
from neo4j import GraphDatabase
from pyvis.network import Network

# Connect to the Neo4j database
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "postgres"))

# Define the Cypher query
cypher_query = """
MATCH (node)-[r]-(n)-[r1]-(n1)
WHERE node.id =~ '(?i).*illegal.*'
RETURN node,r ,n1, r1, n
"""

# Execute the query and retrieve the graph data
with driver.session() as session:
    result = session.run(cypher_query)
    graph_data = [(record["node"]["id"], 
                   record["node"]["type"], 
                   record["n"]["id"], 
                   record["n"]["type"], 
                   record["r"].type,
                   record["n1"]["id"], 
                   record["n1"]["type"], 
                   record["r1"].type) for record in result]

# Create a Pyvis Network object
network = Network(height="500px", width="100%", notebook=True,cdn_resources='remote')

# Define node colors
node_colors = {
    "â\x80\x9cillegalâ": "black",
    "â\x80\x9cIllegal": "black",
    "Dark Web Vendor Illegal Narcotics": "black",
    "Heroin Cocaine Exchange Bitcoin": "black",
    "Mar de la Vida OJSC": "navy",
    "979893388": "green",
    "Oceanfront Oasis Inc Carriers": "orange",
    "8327": "purple",
    "John Gonzalez": "red"
}

# Define node shapes
node_shapes = {
    "â\x80\x9cillegalâ": "square",
    "â\x80\x9cIllegal": "square",
    "Dark Web Vendor Illegal Narcotics": "square",
    "Heroin Cocaine Exchange Bitcoin": "square",
    "Mar de la Vida OJSC": "star",
    "979893388": "star",
    "Oceanfront Oasis Inc Carriers": "star",
    "8327": "star",
    "John Gonzalez":"diamond"
    
}

# Add nodes and edges to the network
for source, source_type,mid, mid_type, relation_mid_type, target, target_type, relation_type   in graph_data:
    network.add_node(source, title=source, shape=node_shapes.get(source,"dot"), color=node_colors.get(source, "lightblue"))
    network.add_node(target, title=target, shape=node_shapes.get(target,"dot"), color=node_colors.get(target, "lightblue"))
    network.add_node(mid, title=mid, shape=node_shapes.get(mid,"dot"), color=node_colors.get(mid, "lightblue"))
    network.add_edge(source, mid, title=relation_mid_type)
    network.add_edge(mid, target, title=relation_type)

# Visualize the network
network.show("6_Illegal_lvl2.html")

# Close the Neo4j driver connection
driver.close()

6_Illegal_lvl2.html


# Gráfico John Gonzalez Hasta nivel 2 (solo con entidades a investigar)

In [135]:
from neo4j import GraphDatabase
from pyvis.network import Network

# Connect to the Neo4j database
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "postgres"))

# Define the Cypher query
cypher_query = """
MATCH (node)-[r1]-(n1)-[r2]-(n2)
WHERE node.id =~ 'John Gonzalez'
AND (n1.id IN ["Mar de la Vida OJSC", "979893388", "Oceanfront Oasis Inc Carriers", "8327"] OR n2.id IN ["Mar de la Vida OJSC", "979893388", "Oceanfront Oasis Inc Carriers", "8327"])
RETURN node,r1,n1,r2,n2
"""

# Execute the query and retrieve the graph data
with driver.session() as session:
    result = session.run(cypher_query)
    graph_data = [(record["node"]["id"], record["node"]["type"], record["n1"]["id"], record["n2"]["id"], record["n1"]["type"], record["n2"]["type"], record["r1"].type,record["r2"].type,) for record in result]

# Create a Pyvis Network object
network = Network(height="500px", width="100%", notebook=True, cdn_resources='remote')

# Define node colors based on id
node_colors = {
    "John Gonzalez": "red",
    "Mar de la Vida OJSC": "navy",
    "979893388": "green",
    "Oceanfront Oasis Inc Carriers": "orange",
    "8327": "purple"}

# Shapes
node_shapes = {
    "Mar de la Vida OJSC": "star",
    "979893388": "star",
    "Oceanfront Oasis Inc Carriers": "star",
    "8327": "star",
    "John Gonzalez":"diamond"
}

# Add nodes and edges to the network
for source, source_type, n1, n2, n1_type, n2_type , r1_type, r2_type in graph_data:
    if n2 in ["Mar de la Vida OJSC", "979893388", "Oceanfront Oasis Inc Carriers", "8327"]:
        network.add_node(source, title=source_type,shape = node_shapes.get(source,"dot"), color=node_colors.get(source, "lightblue"))
        network.add_node(n1, title=n1_type,shape = node_shapes.get(n1,"dot"), color=node_colors.get(n1, "lightblue"))
        network.add_node(n2, title=n2_type,shape = node_shapes.get(n2,"dot"), color=node_colors.get(n2, "lightblue"))
        network.add_edge(source, n1, title=r1_type)
        network.add_edge(n1, n2, title=r2_type)
    elif n1 in ["Mar de la Vida OJSC", "979893388", "Oceanfront Oasis Inc Carriers", "8327"]:
        network.add_node(source, title=source_type,shape = node_shapes.get(source,"dot"), color=node_colors.get(source, "lightblue"))
        network.add_node(n1, title=n1_type,shape = node_shapes.get(n1,"dot"), color=node_colors.get(n1, "lightblue"))
        network.add_edge(source, n1, title=r1_type)
        
# Visualize the network
network.show("9_JG_Entidades.html")

# Close the Neo4j driver connection
driver.close()

9_JG_Entidades.html


# Conexión John Gonzalez Con Ilegalidades

In [129]:
from neo4j import GraphDatabase
from pyvis.network import Network

# Connect to the Neo4j database
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "postgres"))

# Define the Cypher query
cypher_query = """
MATCH (node)-[source_mid]-(mid)-[mid_target]-(target)
WHERE (node.id =~ '(?i).*illegal.*' OR node.id = 'Heroin Cocaine Exchange Bitcoin')  AND target.id = 'John Gonzalez'
RETURN node, source_mid ,mid, mid_target, target
"""

# Execute the query and retrieve the graph data
with driver.session() as session:
    result = session.run(cypher_query)
    graph_data = [(record["node"]["id"], 
                   record["node"]["type"], 
                   record["source_mid"].type,
                   record["mid"]["id"], 
                   record["mid"]["type"], 
                   record["mid_target"].type,
                   record["target"]["id"], 
                   record["target"]["type"] 
                   ) for record in result]

# Create a Pyvis Network object
network = Network(height="500px", width="100%", notebook=True, cdn_resources = 'remote')

# Define node colors
node_colors = {
    "â\x80\x9cillegalâ": "black",
    "â\x80\x9cIllegal": "black",
    "Dark Web Vendor Illegal Narcotics": "black",
    "Heroin Cocaine Exchange Bitcoin": "black",
    "John Gonzalez": "red"
}

# Define node shapes
node_shapes = {
    "â\x80\x9cillegalâ": "square",
    "â\x80\x9cIllegal": "square",
    "Dark Web Vendor Illegal Narcotics": "square",
    "Heroin Cocaine Exchange Bitcoin": "square",
    "John Gonzalez":"diamond"
}

# Add nodes and edges to the network
for source, source_type, source_mid, mid, mid_type, mid_target, target, target_type  in graph_data:
    network.add_node(source, title=source_type, shape = node_shapes.get(source,"dot"), color=node_colors.get(source, "lightblue"))
    network.add_node(mid, title=mid_type, shape = node_shapes.get(mid,"dot"), color=node_colors.get(mid, "lightblue"))
    network.add_node(target, title=target_type, shape = node_shapes.get(target,"dot"), color=node_colors.get(target, "lightblue"))
    network.add_edge(source, mid, title=source_mid)
    network.add_edge(mid, target, title=mid_target)
network.add_edge("Dark Web Vendor Illegal Narcotics", "Heroin Cocaine Exchange Bitcoin", title="partnership")


# Visualize the network
network.show("7_JohnGonzalez_Ilegalidades.html")

# Close the Neo4j driver connection
driver.close()

7_JohnGonzalez_Ilegalidades.html


# ¿Quién es John Gonzalez?

In [139]:
from neo4j import GraphDatabase
from pyvis.network import Network

# Connect to the Neo4j database
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "postgres"))

# Define the Cypher query
cypher_query = """
MATCH (node)-[rel]-(target)
WHERE node.id ='John Gonzalez'
RETURN node, rel, target
"""

# Execute the query and retrieve the graph data
with driver.session() as session:
    result = session.run(cypher_query)
    graph_data = [(record["node"]["id"], 
                   record["node"]["type"],
                   record["rel"].type,
                   record["target"]["id"], 
                   record["target"]["type"] 
                   ) for record in result]

# Create a Pyvis Network object
network = Network(height="500px", width="100%", notebook=True, cdn_resources = 'remote')

# Define node colors
node_colors = {
    "â\x80\x9cillegalâ": "black",
    "â\x80\x9cIllegal": "black",
    "Dark Web Vendor Illegal Narcotics": "black",
    "Heroin Cocaine Exchange Bitcoin": "black",
    "John Gonzalez": "red",
    "organization": "navy",
    "company": "darkgreen",
    "person": "darkorange",
    "political_organization": "lightblue",
    "vessel": "magenta",
    "location": "purple",
    "notype": "darkgrey",
    "event": "cyan",
    "movement": "black",
    "979893388": "green"
}

# Define node shapes
node_shapes = {
    "â\x80\x9cillegalâ": "square",
    "â\x80\x9cIllegal": "square",
    "Dark Web Vendor Illegal Narcotics": "square",
    "Heroin Cocaine Exchange Bitcoin": "square",
    "John Gonzalez":"diamond",
    "Mar de la Vida OJSC": "star",
    "979893388": "star",
    "Oceanfront Oasis Inc Carriers": "star",
    "8327": "star"
}

# Add nodes and edges to the network
for source, source_type, rela, target, target_type  in graph_data:
    network.add_node(source, title=source_type, shape = node_shapes.get(source,"dot"), color=node_colors.get(source, "lightblue"))
    if target in ["Mar de la Vida OJSC", "979893388", "Oceanfront Oasis Inc Carriers", "8327"]:
        network.add_node(target, title=target_type, shape = node_shapes.get(target,"dot"), color=node_colors.get(target, "lightblue"))
    else:
        network.add_node(target, title=target_type, shape = node_shapes.get(target,"dot"), color=node_colors.get(target_type, "lightblue"))
    network.add_edge(source, target, title=rela)
    
# Visualize the network
network.show("8_JohnGonzalez_lvl1.html")

# Close the Neo4j driver connection
driver.close()

8_JohnGonzalez_lvl1.html


In [115]:
for i in range(len(graph_data)):
        print (graph_data[i])

('â\x80\x9cillegalâ', 'notype', '5007554', 'organization', 'partnership')
('â\x80\x9cIllegal', 'notype', '967900443', 'organization', 'partnership')
('â\x80\x9cIllegal', 'notype', 'Odisha Sea Dry dock LLC Consultants', 'notype', 'partnership')
('â\x80\x9cIllegal', 'notype', 'Gary Butler', 'person', 'partnership')
('â\x80\x9cIllegal', 'notype', '59', 'organization', 'partnership')
('â\x80\x9cIllegal', 'notype', '408119', 'organization', 'partnership')
('â\x80\x9cIllegal', 'notype', 'Ryan Garrett', 'person', 'partnership')
('â\x80\x9cIllegal', 'notype', 'Gary Smith', 'person', 'partnership')
('â\x80\x9cIllegal', 'notype', 'Yu gan BV Investment', 'notype', 'partnership')
('â\x80\x9cIllegal', 'notype', '143129355', 'organization', 'partnership')
('â\x80\x9cIllegal', 'notype', '48259', 'organization', 'partnership')
('â\x80\x9cIllegal', 'notype', 'SMILE', 'notype', 'partnership')
('â\x80\x9cIllegal', 'notype', 'SMILE', 'notype', 'partnership')
('â\x80\x9cIllegal', 'notype', '8327', 'organiz